In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# pd.set_option('display.max_colwidth', None)

In [58]:
# Reading the log file
df = pd.read_csv('combined_error_log.csv',index_col=False)

In [59]:
# Seeing data
df.head()

,IP,TIME,PLATFORM,REQ_URL,REQ_FEED,DESC
0,172.29.63.76,17-12-2020 0:0:5,desktop,https://eisamay.indiatimes.com/entertainment/cinema/articlelist/51012377.cms,https://eisamay.indiatimes.com/pwafeeds/sc_gadgetlist.cms?category=mobile&feedtype=sjson&pagetype=widget&perpage=10&sort=latest,Error: Request failed with status code 404
1,172.29.63.76,17-12-2020 0:0:7,desktop,https://eisamay.indiatimes.com/entertainment/cinema/articlelist/51012377.cms,NaN,'Error [ERR_HTTP_HEADERS_SENT]: Cannot set headers after they are sent to the clientat ServerResponse.setHeader (_http_outgoing.js:481:11)at y (/opt/deployment/production/eisweb_63/dist/server.js:1:1085991)at Object.<anonymous> (/opt/deployment/production/eisweb_63/dist/server.js:1:1090471)at tryCatch (/opt/deployment/production/eisweb_63/node_modules/regenerator-runtime/runtime.js:65:40)at Generator.invoke [as _invoke] (/opt/deployment/production/eisweb_63/node_modules/regenerator-runtime/runtime.js:303:22)at Generator.prototype.(anonymous function) [as next] (/opt/deployment/production/eisweb_63/node_modules/regenerator-runtime/runtime.js:117:21)at n (/opt/deployment/production/eisweb_63/dist/server.js:1:1080407)at /opt/deployment/production/eisweb_63/dist/server.js:1:1080553at new Promise (/opt/deployment/production/eisweb_63/node_modules/core-js/modules/es6.promise.js:177:7)at /opt/deployment/production/eisweb_63/dist/server.js:1:1080351'
2,172.29.63.76,17-12-2020 0:0:8,mobile,https://eisamay.indiatimes.com/west-bengal-news/24pargana-news/articlelist/49780592.cms,https://eisamay.indiatimes.com/pwafeeds/sc_gadgetlist.cms?category=mobile&feedtype=sjson&pagetype=widget&perpage=10&sort=latest,Error: Request failed with status code 404
3,172.29.63.76,17-12-2020 0:0:14,mobile,https://eisamay.indiatimes.com/bangla-news-live/-/amp_articleshow/63640426.cms,NaN,'TypeError: Cannot read property 'pagetype' of undefinedat t.value (/opt/deployment/production/eismweb_79/dist/server.js:1:182185)at /opt/deployment/production/eismweb_79/dist/server.js:1:440852at O (/opt/deployment/production/eismweb_79/dist/server.js:1:441056)at e.t.render (/opt/deployment/production/eismweb_79/dist/server.js:1:450327)at e.t.read (/opt/deployment/production/eismweb_79/dist/server.js:1:449906)at renderToString (/opt/deployment/production/eismweb_79/dist/server.js:1:455097)at w (/opt/deployment/production/eismweb_79/dist/server.js:1:1243482)at Object.<anonymous> (/opt/deployment/production/eismweb_79/dist/server.js:1:1247299)at tryCatch (/opt/deployment/production/eismweb_79/node_modules/regenerator-runtime/runtime.js:65:40)at Generator.invoke [as _invoke] (/opt/deployment/production/eismweb_79/node_modules/regenerator-runtime/runtime.js:303:22)'
4,172.29.63.76,17-12-2020 0:0:16,desktop,https://eisamay.indiatimes.com/videos/entertainment/vidyas-kahaani-2-to-clash-with-srk-alias-dear-zindagi-now/videoshow/54488137.cms/feeds/vod_player.cms?autoplay=0&dim=0&msid=54488137&source=article&userInitiated=1&videosection=article-show,NaN,'TypeError: Cannot read property 'components' of undefinedat c (/opt/deployment/production/eisweb_63/dist/server.js:1:1081152)at /opt/deployment/production/eisweb_63/dist/server.js:1:1089765at /opt/deployment/production/eisweb_63/node_modules/react-router/lib/match.js:67:5at /opt/deployment/production/eisweb_63/node_modules/react-router/lib/createTransitionManager.js:72:11at next (/opt/deployment/production/eisweb_63/node_modules/react-router/lib/AsyncUtils.js:52:7)at loopAsync (/opt/deployment/production/eisweb_63/node_modules/react-router/lib/AsyncUtils.js:56:3)at matchRoutes (/opt/deployment/production/eisweb_63/node_modules/react-router/lib/matchRoutes.js:245:29)at Object.match (/opt/deployment/production/eisweb_63/node_modules/react-router/lib/createTransitionManager.js:66:33)at match (/opt/deployment/production/eisweb_63/node_modules/react-router/lib/match.js:56:21)at /opt/deployment/production/eisweb_63/dist/server.js:1:1089641'


In [60]:
# Seeing platform specific
# 71 percent of errors are in desktop in this ip while 28 in mobile
df['PLATFORM'].value_counts(normalize=True)

desktop    0.655512
mobile     0.344488
Name: PLATFORM, dtype: float64

In [61]:
## Seeing feed errors vs others
df[~(df['REQ_FEED'].isnull())].count()[0] / df.shape[0] 
# 41.8 percent feederrors 

0.5031858630710637

In [62]:
def categorize_url(url):
    template_map = {
        'amp_':'amp',
        'articleshow':'articleshow',
        'photoshow':'photoshow',
        'photolist':'photolist',
        'videoshow':'videoshow',
        'articlelist':'articlelist',
        'topics':'topics'
    }
    for template in template_map:
        if template in url:
            return template_map[template]
    return 'others'
df['URL_CATEGORY'] = df['REQ_URL'].apply(categorize_url)


In [63]:
df['URL_CATEGORY'].value_counts()

articlelist    24940
articleshow    15010
amp             5779
videoshow       4815
photoshow       2330
others          1961
photolist        443
topics           123
Name: URL_CATEGORY, dtype: int64

In [64]:
# How i found the categories:
# First inspected the df, found 404 and undefined, started with these as a base along with null
# Characterized others as "others" manually inspected others one by one and
# found: 410 ,403, ['object Object'] timeout
def categorize_error(error):
    #     categories = ['null','undefined','404','410','ERR_HTTP_HEADERS_SENT','9000ms']
    categories = {
       '[object Object]':'[object Object]','undefined':'undefined','404':'404','410':'410','ECONNRESET':'ECONNRESET','ERR_HTTP_HEADERS_SENT':'ERR_HTTP_HEADERS_SENT','9000ms':'timeout'       
    }
    for cat in categories:
        if cat in error:
            return categories[cat]
    return 'other'
            
df['ERROR_TYPE'] = df['DESC'].apply(categorize_error)

In [65]:
df['ERROR_TYPE'].value_counts()

404                      26996
ERR_HTTP_HEADERS_SENT    17928
undefined                 7203
410                       2618
[object Object]            353
other                      185
timeout                    113
ECONNRESET                   5
Name: ERROR_TYPE, dtype: int64

In [66]:
# Very vague type of error
df['DESC'][df['ERROR_TYPE'] == '[object Object]']

107      '[object Object]'
112      '[object Object]'
119      '[object Object]'
125      '[object Object]'
181      '[object Object]'
               ...        
55077    '[object Object]'
55085    '[object Object]'
55264    '[object Object]'
55391    '[object Object]'
55393    '[object Object]'
Name: DESC, Length: 353, dtype: object

In [67]:
# Further inspection reveals that some urls are broken with '-' in between. It will be useful if we categorize these.
df['REQ_URL'][df['ERROR_TYPE'] == '[object Object]']

107                                                                             https://eisamay.indiatimes.com/nation/army-officer-shot-dead-by-jawan-in-uri-sector-in-jammu-and-kashmir/-/jawan/photoshow/59645394.cms
112                                                                    https://eisamay.indiatimes.com/business/business-news/jana-small-finance-bank-provides-9-6-interest-rate-on-fixed-deposit/photoshow/68437855.cms
119                                                                                                    https://eisamay.indiatimes.com/lifestyle/health-fitness/6-proven-health-benefits-of-dates/photoshow/68343345.cms
125                                                                 https://eisamay.indiatimes.com/business/business-news/jana-small-finance-bank-provides-9-6-interest-rate-on-fixed-deposit/fd/photoshow/68437855.cms
181                                                                                                                                     

In [68]:
# Categories to check where we failed in urls
# Manually inspecting, i found '-/' '/-/' and cases like
df['IS_URL_ERROR'] = df['REQ_URL'].apply(lambda x: '/-/' in x or '-/' in x or '%' in x)

> Analyse time of errors to find out if there is a pattern

* **Following are the amp urls where the url is correctly formed, but still it fails** 

In [72]:
df['REQ_URL'][(df['URL_CATEGORY'] == 'amp') & ~(df['IS_URL_ERROR'])].head(20)

18                                                          https://eisamay.indiatimes.com/tech/mobile-review/amp_articlelist/64760842.cms
28                                                       https://eisamay.indiatimes.com/entertainment/tv-news/amp_articlelist/15900880.cms
29                                             https://eisamay.indiatimes.com/west-bengal-news/24pargana-news/amp_articlelist/49780592.cms
35                                                                             https://eisamay.indiatimes.com/amp_articlelist/50119970.cms
37             https://eisamay.indiatimes.com/lifestyle/sex/bizarre-sex-rules-of-various-countries-in-world/amp_articleshow/photo_77053620
38                                                    https://eisamay.indiatimes.com/lifestyle/health-fitness/amp_articlelist/15992584.cms
46                                             https://eisamay.indiatimes.com/west-bengal-news/bardhaman-news/amp_articlelist/51647343.cms
55             https://eisa